# Plot groups' analysis
## Set parameters

In [ ]:
experiment_directory = "experiment"
proof_directory = "proof"
rebased_directory = "rebased_on_mjd"
use_local_data = True                               # if False, it tries to read the data on the laboratory's server
# ###################################### LOCAL DIRECTORIES ######################################
experiment_root = f"./data/experiments/soumnya/{experiment_directory}/"
proof_root      = f"./data/experiments/soumnya/{proof_directory}/"
rebased_root    = f"./data/experiments/soumnya/{rebased_directory}/"

In [ ]:
# ####################################### MAC DIRECTORIES #######################################
experiment_root = "/Volumes/Ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
proof_root = "/Volumes/Ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/proof/"
rebased_root = "/Volumes/Ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/rebased_on_mjd/"

In [ ]:
# ###################################### GNOME DIRECTORIES ######################################
experiment_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
proof_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/proof/"
rebased_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/rebased_on_mjd/"

In [ ]:
# ###################################### LOCAL DIRECTORIES ######################################
experiment_root = "./data/experiments/soumnya/experiment/"
proof_root = "./data/experiments/soumnya/proof/"
rebased_root = "./data/experiments/soumnya/rebased_on_mjd/"

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
branches_to_exclude = ["retina", "VS", "grv", "fiber tracts", "CB"]

# ###################################### DATA LOAD OPTIONS ######################################
area_key = "Area um^2"
tracer_key = "Num AF647"
marker = "CFos"
rebase_regions_on = "major divisions"                           # "major divisions" or "summary structures"

# ###################################### DATA SAVE OPTIONS ######################################
animal_brain_mode = "sum"                                       # available options are: 'sum', 'mean'/'avg', 'std', 'variation'/'cvar'
save_animals = True
save_groups = True

# ###############################################################################################

## Script's code
run all cell below

In [ ]:
import BraiAn
import os
import sys
from typing import List

In [ ]:
if not use_local_data:
    match sys.platform:
        case "darwin":
            mnt_point = "/Volumes/Ricerca/"
            
        case "linux":
            mnt_point = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/"
        case _:
            raise Exception(f"Can't find the 'Ricerca' folder in the server for '{sys.platform}' operative system. Please report the developer (Carlo)!")
    if not os.path.isdir(mnt_point):
        raise Exception(f"Could not read '{mnt_point}'. Please be sure you are connected to the server.")
    experiment_root = os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", "Mathias", "soumnya", "data", experiment_directory)
    proof_root      = os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", "Mathias", "soumnya", "data", proof_directory)
    rebased_root    = os.path.join(mnt_point, "Lab Matteoli", "Silva", "collaborations", "Mathias", "soumnya", "data", rebased_directory)

proof_input_path        = os.path.join(proof_root, "QuPath_output")
experiment_input_path   = os.path.join(experiment_root, "QuPath_output")
rebased_output_path     = os.path.join(rebased_root, "QuPath_output")

assert not os.path.isdir(rebased_output_path), "rebased directory already exists!"

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)
match rebase_regions_on:
    case "major divisions":
        AllenBrain.select_regions(BraiAn.MAJOR_DIVISIONS)
    case "summary structures":
        AllenBrain.select_from_csv("./data/AllenSummaryStructures.csv")
    case _:
        raise Exception(f"Invalid value '{rebase_regions_on}' for rebase_regions_on")
        
selected_regions = AllenBrain.get_selected_regions()
print(f"You selected {len(selected_regions)} regions.")

In [ ]:
proof_animal_dirs = [d for d in os.listdir(proof_input_path) if os.path.isdir(os.path.join(proof_input_path, d)) and not d.startswith(".")]
proof_slices = [BraiAn.SlicedBrain(animal_dir,
                                    os.path.join(proof_input_path, animal_dir),
                                    AllenBrain,
                                    area_key,
                                    tracer_key,
                                    marker,
                                    area_units="µm2")
                    for animal_dir in proof_animal_dirs]
print(f"Imported all brain slices from {str(len(proof_animal_dirs))} animals of 'proof'.")

experiment_animal_dirs = [d for d in os.listdir(experiment_input_path) if os.path.isdir(os.path.join(experiment_input_path, d)) and not d.startswith(".")]
experiment_slices = [BraiAn.SlicedBrain(animal_dir,
                                    os.path.join(experiment_input_path, animal_dir),
                                    AllenBrain,
                                    area_key,
                                    tracer_key,
                                    marker,
                                    area_units="µm2")
                    for animal_dir in experiment_animal_dirs]
print(f"Imported all brain slices from {str(len(experiment_animal_dirs))} animals of 'experiment'.")

In [ ]:
missing_in_experiment = [animal for animal in proof_animal_dirs if animal not in experiment_animal_dirs]
missing_in_proof = [animal for animal in experiment_animal_dirs if animal not in proof_animal_dirs]
print(f"Animals missing in 'experiment': {', '.join(missing_in_experiment)}")
print(f"Animals missing in 'proof': {', '.join(missing_in_proof)}")

In [ ]:
# NOTE: brains are being written WITH Left/Right discrimination
# If you desire to save them without, call AnimalBrain with hemisphere_distinction=False

min_slices = 3
proof_sum_brains: List[BraiAn.AnimalBrain] = [BraiAn.AnimalBrain(sliced_brain, mode=animal_brain_mode, min_slices=min_slices, hemisphere_distinction=False) for sliced_brain in proof_slices]
experiment_sum_brains: List[BraiAn.AnimalBrain] = [BraiAn.AnimalBrain(sliced_brain, mode=animal_brain_mode, min_slices=min_slices, hemisphere_distinction=False) for sliced_brain in experiment_slices]

In [ ]:
def get_rebase_coefficient(proof_brain: BraiAn.AnimalBrain, experiment_brain: BraiAn.AnimalBrain):
    assert proof_brain.name == experiment_brain.name, f"The rebase coefficient can't be computed between two different animals '{proof_brain.name}' and '{experiment_brain.name}'"
    regions_in_both = list(set(proof_brain.data.index).intersection(set(experiment_brain.data.index)))
    proof_density       = (     proof_brain.data[marker] /       proof_brain.data.area)[regions_in_both].mean()
    experiment_density  = (experiment_brain.data[marker] /  experiment_brain.data.area)[regions_in_both].mean()
    return proof_density / experiment_density

In [ ]:
proof_selected_regions = [BraiAn.AnimalBrain.filter_selected_regions(brain, AllenBrain) for brain in proof_sum_brains if brain.name not in missing_in_experiment]
proof_selected_regions = [brain for brain in sorted(proof_selected_regions, key=lambda x: x.name)]
experiment_selected_regions = [BraiAn.AnimalBrain.filter_selected_regions(brain, AllenBrain) for brain in experiment_sum_brains if brain.name not in missing_in_proof]
experiment_selected_regions = [brain for brain in sorted(experiment_selected_regions, key=lambda x: x.name)]

coefficients = {}
for proof_brain, experiment_brain in zip(proof_selected_regions, experiment_selected_regions):
    rebase_coefficient = get_rebase_coefficient(proof_brain, experiment_brain)
    coefficients[proof_brain.name] = rebase_coefficient

In [ ]:
proof_brain, experiment_brain = proof_selected_regions[0].data, experiment_selected_regions[0].data
regions_in_both = list(set(proof_brain.index).intersection(set(experiment_brain.index)))
proof_density       = (     proof_brain[marker] /       proof_brain.area)[regions_in_both].mean()
experiment_density  = (experiment_brain[marker] /  experiment_brain.area)[regions_in_both].mean()

rebase_coefficient = proof_density / experiment_density
rebase_coefficient, rebase_coefficient * experiment_density, proof_density, ((experiment_brain[marker] * rebase_coefficient) / experiment_brain.area)[regions_in_both].mean()

In [ ]:
import pandas as pd
import shutil

for animal_dir, coefficient in coefficients.items():
    experiment_results_dir = os.path.join(experiment_input_path, animal_dir, "results")
    experiment_regions_to_exclude_dir = os.path.join(experiment_input_path, animal_dir, "regions_to_exclude")
    output_results_dir = os.path.join(rebased_output_path, animal_dir, "results")
    os.makedirs(output_results_dir, exist_ok=True)
    output_regions_to_exclude_dir = os.path.join(rebased_output_path, animal_dir, "regions_to_exclude")
    os.makedirs(output_regions_to_exclude_dir, exist_ok=True)
    for slice_file in os.listdir(experiment_results_dir):
        if not slice_file.endswith("_regions.txt"):
            continue
        slice_name = slice_file.replace("_regions.txt", "")
        experiment_slice_results_file = os.path.join(experiment_results_dir, slice_file)
        experiment_slice_results = pd.read_csv(experiment_slice_results_file, sep="\t").drop_duplicates()
        experiment_slice_results[tracer_key] *= coefficient # WE REBASE THE SLICE
        experiment_slice_results["Num Detections"] *= coefficient
  
        experiment_slice_regions_to_exclude_file = os.path.join(experiment_regions_to_exclude_dir, f"{slice_name}_regions_to_exclude.txt")
        output_slice_regions_to_exclude_file = os.path.join(output_regions_to_exclude_dir, f"{slice_name}_regions_to_exclude.txt")
        shutil.copy2(experiment_slice_regions_to_exclude_file, output_slice_regions_to_exclude_file)
        output_slice_results_file = os.path.join(output_results_dir, f"{slice_name}_regions.txt")
        experiment_slice_results.to_csv(output_slice_results_file, sep="\t", mode="w")

In [ ]:
# We have to think very well what to do with rebase_coefficient. Can we use the rebase coefficient, computed on density, on marker numbers (surely not on area)?
# I would say yes.

In [ ]:
import importlib
importlib.reload(BraiAn.sliced_brain)
importlib.reload(BraiAn.animal_brain)
importlib.reload(BraiAn)